In [69]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import os
import zipfile

In [54]:
from google.colab import files
uploaded = files.upload()

# 예를 들어 업로드한 파일 이름이 아래와 같다면
df = pd.read_csv('JSTdatasetR3.CSV')

Saving JSTdatasetR3.CSV to JSTdatasetR3 (5).CSV


나라별로 변수 생성하기

In [68]:
import pandas as pd

# 1. 파생 변수 생성 함수
def create_var(group):
    group = group.sort_values('year')
    new_vars = pd.DataFrame(index=group.index)
    new_vars['country'] = group['country'].values
    new_vars['year'] = group['year'].values

    new_vars['yield_curve_slope'] = group['ltrate'] - group['stir']
    new_vars['credit_gdp_2y_diff'] = (group['tloans'] / group['gdp']).diff(2) * 100
    new_vars['cpi_2y_growth'] = (group['cpi'] / group['cpi'].shift(2) - 1) * 100
    new_vars['debt_service_ratio'] = (group['stir'] * group['tloans'] / group['gdp']).diff(2) * 100
    new_vars['consumption_growth'] = (group['rconpc'] / group['rconpc'].shift(2) - 1) * 100
    new_vars['investment_2y_diff'] = group['iy'].diff(2) * 100
    new_vars['debt_gdp_2y_diff'] = group['debtgdp'].diff(2) * 100
    new_vars['money_gdp_2y_diff'] = (group['money'] / group['gdp']).diff(2) * 100
    new_vars['stocks_2y_growth'] = (group['stocks'] / group['stocks'].shift(2) - 1) * 100
    new_vars['ca_gdp_2y_diff'] = (group['ca'] / group['gdp']).diff(2) * 100
    new_vars['household_loans_2y_diff'] = (group['thh'] / group['gdp']).diff(2) * 100
    new_vars['business_loans_2y_diff'] = (group['tbus'] / group['gdp']).diff(2) * 100
    new_vars['house_price_2y_growth'] = (group['hpnom'] / group['hpnom'].shift(2) - 1) * 100

    return new_vars

# 2. 국가별 변수 생성 및 저장
varData_dict = {}

for country, group in df.groupby('country'):
    country_vars = create_var(group)
    varData_dict[country] = country_vars

    # 파일 이름에서 공백 제거
    safe_country = country.replace(' ', '_')
    filename = f'{safe_country}_vars.csv'
    country_vars.to_csv(filename, index=False)


In [70]:
with zipfile.ZipFile('country_vars.zip', 'w') as zipf:
    for country in varData_dict:
        safe_country = country.replace(' ', '_')
        filename = f'{safe_country}_vars.csv'
        zipf.write(filename)

# 다운로드
from google.colab import files
files.download('country_vars.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
# 인덱스 정리 및 합치기
for country in varData_dict:
    varData_dict[country] = varData_dict[country].reset_index(drop=True)

# 합치기
combined_df = pd.concat(varData_dict.values(), ignore_index=True)

# 저장
combined_df.to_csv('all_countries_vars.csv', index=False)

from google.colab import files
files.download('all_countries_vars.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>